<a name="TOP"></a>

# Lab Outline

**TODO**:
- Republish on github - cleaner version

## Online

#### This document
You can find this document online as
- a PDF file or as
- a [Jupyter](http://www.jupyter.org) notebook at [2016-Feb_Docker_Build_Lab](https://github.com/mjbright/jupyter_notebooks/blob/master/2016-Feb_Docker_Build_Lab/).

This notebook is runnable in a Jupyter installation with the bash_kernel installed.

<br/><br/><br/>
<center><font size=-1 color="#77f">
    <i class="fa-github fa-2x fa"> On Github: [mjbright](https://github.com/mjbright)</i>,&nbsp;&nbsp;
    <i class="fa-twitter fa-2x fa"> On Twitter: [@mjbright](http://twitter.com/mjbright)</i>,&nbsp;&nbsp;
    <i class="fa-user fa-2x fa"> Blog: [mjbright.github.io](http://mjbright.github.io)</i><br/>
    <i class="fa-envelope fa-2x fa"> mjbrightfr AT gmail</i>
</font></center>


## Lab-Description
[TOP](#TOP)
- [Lab-p1](#Lab-p1)
- [Lab-p2](#Lab-p2)
  - [Rails example with Compose](#Rails-Example-with-Compose)
  - [Building Docker with Docker](#Building-Docker-with-Docker)

[References](#References)

Lab to be generated as a Jupyter Notebook (based on bash kernel - or bash metakernel)

- Lab details
  - Nature
  - Duration
  - Planning ( Dates )
  - Audience
  
### Lab setup

** TODO: add use of simple web server, handlebar.js-template, angular-template, go-web server, ...**
** TODO: go web server as miniscule image ???? **

- **configuration to use Yann's environment**
- Removal of images
  - base: just large images present (to minimize network bandwidth)
    - alpine, ubuntu, language e.g. Python/Nodejs?), Golang, Raspi ??
- Creation of lab accounts (**hpe_tss_lab**) on DockerHub, Github and GMail
  - Attendees must take care to name their images/repos according to their pod number, e.g. **userN**


### Overall description of the lab steps

**NOTE: All lab steps can be considered optional, attendees may perform them in order, or jump to the section of interest to them (to get to the more complicated steps)**

# Lab-p1
## A refresh on Docker concepts

Look at what docker version you are running.
Note that the 'docker version' command reports the local client version as well as the server (docker engine) version.

In [1]:
docker version

Client:
 Version:      1.10.0-rc2
 API version:  1.22
 Go version:   go1.5.3
 Git commit:   c1cdc6e
 Built:        Wed Jan 27 22:31:21 2016
 OS/Arch:      linux/amd64

Server:
 Version:      1.10.0-rc2
 API version:  1.22
 Go version:   go1.5.3
 Git commit:   c1cdc6e
 Built:        Wed Jan 27 22:31:21 2016
 OS/Arch:      linux/amd64


### Images are image layers

Remember that when we talk of a container image it is really a collection of image layers.

The docker info command provides information about the docker engine, see below.

In [12]:
docker info

Containers: 1
 Running: 0
 Paused: 0
 Stopped: 1
Images: 2
Server Version: 1.10.0-rc2
Storage Driver: aufs
 Root Dir: /var/lib/docker/aufs
 Backing Filesystem: extfs
 Dirs: 12
 Dirperm1 Supported: false
Execution Driver: native-0.2
Logging Driver: json-file
Plugins: 
 Volume: local
 Network: host bridge null
Kernel Version: 3.13.0-55-generic
Operating System: Ubuntu 14.04.2 LTS
OSType: linux
Architecture: x86_64
CPUs: 1
Total Memory: 1.955 GiB
Name: vagrant-ubuntu-trusty-64
ID: 6LRX:E4PK:3EBE:MHJE:TJHR:NVS6:5POR:YS4C:WIWG:7F5J:Y6FU:4IZE


But if we look at the number of containers and images, the number of images it is not the same as provided above.
Why do you think that is?

First let's list the number of running and number of stopped containers

**NOTE: the value on your system will be different**

In [13]:
docker ps | tail -n +2 | wc -l # Number of running containers ('tail -n +2' excludes the header line)

0


In [14]:
docker ps -a | tail -n +2 | wc -l # Number of stopped/running containers ('tail -n +2' excludes the header line)

1


We can see that the number of containers reported by docker info correctly reports the number of total containers, running or not

But listing images gives a different value from the 'docker info' value

In [15]:
docker images | tail -n +2 | wc -l

2


That is because there are many intermediate image layers which are not normally listed.
But we can list those layers using the '-a' option and now we see a number close to the value from 'docker info'.

(We will see later how the 'docker history' command allows us to see how the layers were created).

In [9]:
docker images -a | tail -n +2 | wc -l  # The number of image layers+1 (inc. header line)

1


Images can include 1 static binary file or more and can even include a whole distribution.
Launching a container launches a single process within that container - which may in turn span other child processes.

Let us look at an extremely small image to have an idea just how small an execitable image can be.
Docker provide an official 'hello-world' image which simply echoes some output to the console.

Let's run that image to see and then investigate the image.
First let's search for the image; we see that the first image is 'hello-world' which is an official build

In [10]:
docker search hello-world

NAME                                     DESCRIPTION                                     STARS     OFFICIAL   AUTOMATED
hello-world                              Hello World! (an example of minimal Docker...   47        [OK]       
tutum/hello-world                        Image to test docker deployments. Has Apac...   19                   [OK]
marcells/aspnet-hello-world              ASP.NET vNext - Hello World                     2                    [OK]
sbasyal/java-hello-world                                                                 1                    [OK]
carinamarina/hello-world-app             This is a sample Python web application, r...   1                    [OK]
vegasbrianc/docker-hello-world                                                           1                    [OK]
carinamarina/hello-world-web             A Python web app, running on port 5000, wh...   1                    [OK]
bencampbell/hello-world                  First automated build.                

Let's now run that image

In [11]:
docker run hello-world

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world


Digest: sha256:8be990ef2aeb16dbcb9271ddfe2610fa6658d13f6dfb8bc72074cc1ca36966a7
Status: Downloaded newer image for hello-world:latest

Hello from Docker.
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker Hub account:
 https://hub.docker.com

For more examples and ideas, visit:
 https://

If it took a while to run, this was due to the time needed to download the image before running it - see above.

Try the command a second time to see how it runs instantaneously as there is no need to download the image which already exists locally on the 'docker engine'.

In [16]:
docker run hello-world


Hello from Docker.
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker Hub account:
 https://hub.docker.com

For more examples and ideas, visit:
 https://docs.docker.com/userguide/



Let us inspect the image.  We see that the file is only 960 bytes large, it must be machine code to print out the text.  So we see that an image can be really very small

In [18]:
docker images hello-world

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
hello-world         latest              690ed74de00f        3 months ago        960 B


We can also inspect the image with the history command to see how it was constructed.

Note that history shows the image layers in reverse order, latest first.

From the below command we can see that the image was created from only 2 image layers.

The image was built simply by copying in a binary executable and then specifying the default command to invoke when the image is run.

In [19]:
docker history hello-world

IMAGE               CREATED             CREATED BY                                      SIZE                COMMENT
690ed74de00f        3 months ago        /bin/sh -c #(nop) CMD ["/hello"]                0 B                 
<missing>           3 months ago        /bin/sh -c #(nop) COPY file:1ad52e3eaf4327c8f   960 B               


## Creating a small C Docker image

In this example we show how we can create a Docker image from a statically-linked binary.

<font size=+1 color="#77f">
<b>The goal of this step is to show that we do not need an Operating System image for a Docker container.</b>
</font>

All we need is a self-contained binary - i.e. statically linked binary.

Of course a dynamically linked binary could also be used, but in this case it's more complicated as you would have to manually add all it's dependent libraries.  Let's let gcc to do that work for us!

This section comprises 2 things
- A Dockerfile to build our image from a static binary
  Note that it starts with "FROM scratch".
  Scratch is a special 'empty' image
- helloFromDocker.c
  

In [22]:
cd ~/src/createTinyC/
cat Dockerfile


FROM scratch
MAINTAINER "Docker Build Lab" <dockerlab@mjbright.net>

ADD ./helloFromDocker.exe /helloFromDocker.exe
CMD ["/helloFromDocker.exe"]



So first let's build our static binary



In [ ]:
gcc -static helloWorld.c -o helloWorld.exe

Now let's build our image containing this binary

In [25]:
docker build -t lab/c_prog .


Step 1 : FROM scratch
 ---> 
Step 2 : MAINTAINER "Docker Build Lab" <dockerlab@mjbright.net>
 ---> Using cache
 ---> 736216a2a3eb
Step 3 : ADD ./helloWorld.exe /helloWorld.exe
 ---> 0a40a3444cb7
Removing intermediate container f9f8a4ef27aa
Step 4 : CMD /helloWorld.exe
 ---> Running in 98ce8211fea6
 ---> 0932ac1b6d79
Removing intermediate container 98ce8211fea6
Successfully built 0932ac1b6d79


In [28]:
docker images lab/c_prog

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
lab/c_prog          latest              0932ac1b6d79        33 seconds ago      877.2 kB


And now let's run that image

In [ ]:
docker run lab/c_prog

## Creating a small Go Docker image

That's fine, but isn't Go taking over the world as a systems language?
Docker, Kubernetes, LXD, Rocket, ... many new tools are being written in Go.

Let's see how we can do the same exercise but building a Go statically-linked binary.

<font size=+1 color="#77f">
<b>The goal of this step is as the previous step (building an image from a single statically-linked binary) but using Go.</b>
</font>

We will do this without directly 'installing a Go compiler'

xxxx


  

Hello World!!


In [16]:
docker history mjbright/c_progdd

Error response from daemon: no such id: mjbright/c_prog


Let's pull and examine the official 'Docker Language Stack' image of Node.js

In [17]:
time docker pull node

Using default tag: latest
latest: Pulling from library/node







Digest: sha256:3b69202057d1f98532e2d9952a445addc26a859cb95870384db24499a8a2331f
Status: Downloaded newer image for node:latest

real	0m43.570s
user	0m0.093s
sys	0m0.060s


In [35]:
docker images node

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
node                latest              0869e88f8617        10 days ago         643 MB


In [19]:
docker history node

IMAGE               CREATED             CREATED BY                                      SIZE                COMMENT
0869e88f8617        10 days ago         /bin/sh -c #(nop) CMD ["node"]                  0 B                 
<missing>           10 days ago         /bin/sh -c curl -SLO "https://nodejs.org/dist   36.39 MB            
<missing>           10 days ago         /bin/sh -c #(nop) ENV NODE_VERSION=5.5.0        0 B                 
<missing>           10 days ago         /bin/sh -c #(nop) ENV NPM_CONFIG_LOGLEVEL=inf   0 B                 
<missing>           10 days ago         /bin/sh -c set -ex   && for key in     9554F0   51.75 kB            
<missing>           3 weeks ago         /bin/sh -c apt-get update && apt-get install    314.6 MB            
<missing>           3 weeks ago         /bin/sh -c apt-get update && apt-get install    122.6 MB            
<missing>           3 weeks ago         /bin/sh -c apt-get update && apt-get install    44.27 MB            
<missing>   

Let's pull and examine the official 'Docker Language Stack' image of Python

Note how the earliest image layers (at the bottom of the list) have the same image ids
as the earliest image layers of the Node;js image.

So we can see that they were both created from the same base.

In [20]:
time docker pull python

Using default tag: latest
latest: Pulling from library/python








Digest: sha256:4651b83dd903ce78b1c455794f63d4108d9469a6c7fe97cd07d08a77b7e72435
Status: Downloaded newer image for python:latest

real	0m12.217s
user	0m0.060s
sys	0m0.026s


In [34]:
docker images python

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
python              latest              93049cc049a6        4 days ago          689.1 MB


In [22]:
docker history python

IMAGE               CREATED             CREATED BY                                      SIZE                COMMENT
93049cc049a6        4 days ago          /bin/sh -c #(nop) CMD ["python3"]               0 B                 
<missing>           4 days ago          /bin/sh -c cd /usr/local/bin  && ln -s easy_i   0 B                 
<missing>           4 days ago          /bin/sh -c set -ex  && gpg --keyserver ha.poo   81.53 MB            
<missing>           4 days ago          /bin/sh -c #(nop) ENV PYTHON_PIP_VERSION=7.1.   0 B                 
<missing>           4 days ago          /bin/sh -c #(nop) ENV PYTHON_VERSION=3.5.1      0 B                 
<missing>           4 days ago          /bin/sh -c #(nop) ENV GPG_KEY=97FC712E4C024BB   0 B                 
<missing>           4 days ago          /bin/sh -c #(nop) ENV LANG=C.UTF-8              0 B                 
<missing>           4 days ago          /bin/sh -c apt-get purge -y python.*            978.7 kB            
<missing>   

#### Dockerfile

Images are built from Dockerfiles which contain a series of commands used to build up a docker image.
Note that each command in the Dockerfile results in a new image layer being created, no matter how trivial the command - even ENV "commands" create a nw image layer.

In the following lab we will see how images can be built systematically from a Dockerfile using the 'docker build' command.

#### DockerHub

When we pull an image we pull it from a Docker Registry.  The [DockerHub](https://hub.docker.com/) is a free to use Docker registry allowing to store your own image files (which are publicly available unless you pay for your account) and to pull other image files of other users or officially provided images.

You can create images either by
- building them from a Dockerfile (thus in a **repeatable** manner)
- building them manually by modifying a running container and *'commit'*ing it's state

The DockerHub contains images which may be
- **Automated builds** (built from a git repository)
  - Such builds are usually built from an open-source git repo and so are called **Trusted huilds** because the source code is available.
    *Note:* The github repo may contain binary files though
- **Official builds** are builds which are builds provided by partners or by Docker themselves

Other images may exist in the hub but their origin is unknown and so represent a security risk.

It is possible to search the DockerHub, or another Docker Registry, using the 'docker search' command with appropriate options.  Other companies offer their own Docker Registry which may be freely accessible e.g. RedHat, internal to a company e.g. HPE IT, or available as part of a paid for service e.g. IBM or Amawon Web Services ECR.


# Lab-p2
## Docker Builds

## Simple Docker Build

Let's start by creating a minimal Dockerfile which builds on the Python image we downloaded earlier

In [23]:
mkdir /root/test
cd /root/test

cat > Dockerfile.basic <<EOF
#
# Dockerfile to demonstrate the simplest build
#
FROM python

MAINTAINER "xxx"

RUN python --version

CMD bash

EOF

    - First see what images are present
    - Build a simple image based on ... medium size
    - Observe build process
    - Modify Dockerfile
    - Rebuild - observe caching
    - Move order in Dockerfile
    - Rebuild
    - Rebuild with no caching
    - Perform Docker image
    - Perform docker image -a
    - Perform docker history

In [24]:
ls -altr Dockerfile.basic

-rw-r--r-- 1 root root 114 Jan 31 19:57 Dockerfile.basic


We can now build a new image using this dockerfile using the below command where
- we explicitly select the Dockerfile.basic which we just created
- we specify the current directory as the context for the build (any ADD/COPY or Dockerfile files will be sourced from here)
- we specify the specific tag to se for the generated image as "*lab/basic*"

In [25]:
docker build -f Dockerfile.basic -t lab/basic . 


Step 1 : FROM python
 ---> 93049cc049a6
Step 2 : MAINTAINER "xxx"
 ---> Running in 9eb120f73cee
 ---> 220d1f82b3bb
Removing intermediate container 9eb120f73cee
Step 3 : RUN python --version
 ---> Running in c758ecd28ca6
Python 3.5.1
 ---> efe81c89a2d7
Removing intermediate container c758ecd28ca6
Step 4 : CMD bash
 ---> Running in a52aed9484e8
 ---> a7d63cd9ba3c
Removing intermediate container a52aed9484e8
Successfully built a7d63cd9ba3c


Note that during the build, the RUN commands are actually run.

They are used to build up this new image.

In this case we echo the 'Python' version string during the build process.

You can see the available options to the build command by issuing
 'docker build --help'

In [26]:
docker build --help


Usage:	docker build [OPTIONS] PATH | URL | -

Build an image from a Dockerfile

  --build-arg=[]                  Set build-time variables
  --cpu-shares                    CPU shares (relative weight)
  --cgroup-parent                 Optional parent cgroup for the container
  --cpu-period                    Limit the CPU CFS (Completely Fair Scheduler) period
  --cpu-quota                     Limit the CPU CFS (Completely Fair Scheduler) quota
  --cpuset-cpus                   CPUs in which to allow execution (0-3, 0,1)
  --cpuset-mems                   MEMs in which to allow execution (0-3, 0,1)
  --disable-content-trust=true    Skip image verification
  -f, --file                      Name of the Dockerfile (Default is 'PATH/Dockerfile')
  --force-rm                      Always remove intermediate containers
  --help                          Print usage
  --isolation                     Container isolation level
  -m, --memory                    Memory limit
  --memory-swap       

We can see all the images available using the
 'docker images' command
 
but if there are many, how do we see just our newly-created image?

You can see the available options to the images command by issuing
 'docker images --help'

In [27]:
docker images --help


Usage:	docker images [OPTIONS] [REPOSITORY[:TAG]]

List images

  -a, --all          Show all images (default hides intermediate images)
  --digests          Show digests
  -f, --filter=[]    Filter output based on conditions provided
  --format           Pretty-print images using a Go template
  --help             Print usage
  --no-trunc         Don't truncate output
  -q, --quiet        Only show numeric IDs


So you can see your newly built 'lab/basic' with the following command:

In [28]:
docker images lab/basic

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
lab/basic           latest              a7d63cd9ba3c        43 seconds ago      689.1 MB


Note that if you rerun the build command, the build should run faster, you will notice how build steps recognize that this step has already been performed and so will use the image layer already available in the local cache.

In [29]:
docker build -f Dockerfile.basic -t lab/basic . 


Step 1 : FROM python
 ---> 93049cc049a6
Step 2 : MAINTAINER "xxx"
 ---> Using cache
 ---> 220d1f82b3bb
Step 3 : RUN python --version
 ---> Using cache
 ---> efe81c89a2d7
Step 4 : CMD bash
 ---> Using cache
 ---> a7d63cd9ba3c
Successfully built a7d63cd9ba3c


Now let us see what happens if we modify our Dockerfile, by inserting a line, such as defining an environment variable.

We will use the same Dockerfile, but this time we will insert an "ENV" line


In [30]:
cd /root/test/

cat > Dockerfile.basic <<EOF
#
# Dockerfile to demonstrate the simplest build (with ENV line added)
#
FROM python

MAINTAINER "xxx"

RUN python --version

ENV myvar "this will force rebuilding from here on"

CMD bash

EOF

This time when we build the image we will see that the addition of a line between the "RUN" line and the "CMD" line forces rebuild of subsequent image layers.

**We see 'Using cache' for Step 2 and 3 only**

In [31]:
docker build -f Dockerfile.basic -t lab/basic . 


Step 1 : FROM python
 ---> 93049cc049a6
Step 2 : MAINTAINER "xxx"
 ---> Using cache
 ---> 220d1f82b3bb
Step 3 : RUN python --version
 ---> Using cache
 ---> efe81c89a2d7
Step 4 : ENV myvar "this will force rebuilding from here on"
 ---> Running in 5f7d2a142e79
 ---> c093c7189744
Removing intermediate container 5f7d2a142e79
Step 5 : CMD bash
 ---> Running in 5775938e4f06
 ---> a192650246e9
Removing intermediate container 5775938e4f06
Successfully built a192650246e9


Similarly we can force to not use the cache with the --no-cache option.

This could be useful if we suspect the caching is not working properly due to some external change.

In [32]:
docker build --no-cache=true -f Dockerfile.basic -t lab/basic . 


Step 1 : FROM python
 ---> 93049cc049a6
Step 2 : MAINTAINER "xxx"
 ---> Running in f66f3fcb71eb
 ---> 570f076fda61
Removing intermediate container f66f3fcb71eb
Step 3 : RUN python --version
 ---> Running in 2f15a463d0b3
Python 3.5.1
 ---> 77a14de84a88
Removing intermediate container 2f15a463d0b3
Step 4 : ENV myvar "this will force rebuilding from here on"
 ---> Running in 9b65e2802ff8
 ---> 21fd1676d48b
Removing intermediate container 9b65e2802ff8
Step 5 : CMD bash
 ---> Running in b645658fa73d
 ---> 5477a1cdda04
Removing intermediate container b645658fa73d
Successfully built 5477a1cdda04


In [33]:
docker images lab/basic

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
lab/basic           latest              5477a1cdda04        12 seconds ago      689.1 MB


  
## More complex Docker Builds
    - extending a Dockerfile ("inheritance")

  - Building from different base images
    - e.g. from Alpine

  - Build from Language Stack
    - e.g. build a node application from Node language stack
    - e.g. build a Go   application from Go   language stack

  - Availability of diverse (official/trusted) images
    - Do some MongoDB or something ...
    - simple web server, multi-tier??

- #### Docker Small Builds
  - Build a small image
    Miniature web server?

  - Build a small Go image (using a Go container to build)
    - Show how image is small
    - underline how we didn't need to install a compiler natively
    - Build an incremental image based on this

  - Perform a cross-compilation for Raspberry Pi (need a large image pre-loaded)
    - Perform file on a.out to show architecture
    - not yet a way of seeing architecture of image (either locally or on DockerHub)
    - pull/run an ARM docker image (smallest Hypriot)

# Docker Automated Builds
  - Create an account on DockerHub if necessary
    - **OR** reuse
      - Your own account
      - or reuse lab account **hpe_tss_lab**
    - Push a **small** image to DockerHub
      - if using lab account - name your image as **userN** where N is your pod number
    - Pull the image
    - Look at GMail account ?? (optional - confusing, conflicting ... in separate browser)

  - Create an automated build on DockerHub
    - Create a github account if necessary
      - **OR** reuse
        - Your own account
        - or reuse lab account **hpe_tss_lab**
    - Create a repo
      - if using lab account - name your repo as **userN** where N is your pod number
    - Checkout repo
    - Create a Dockerfile, complete it
    - commit/push repo
    - On DockerHub create automated build
    - Perform initial build
    - See "build log"
    - Modify Dockerfile
    - commit/push repo
    - See "build log"

# Cowsay example
<a href="#TOP">TOP</a>

This example heavily inspired from this article [Containers for Continuous Delivery](http://fredhutchio.github.io/containers-ci/) ... 

<font size=+1 color="#77f">
<b>The goal of this step is ...</b>   
</font>

In [ ]:
mkdir -p /root/src/cowsay
cd /root/src/cowsay

git clone https://github.com/matsen/cowsay-build-env.git
cd cowsay-build-env/

In [6]:
cd /root/src/cowsay
cd cowsay-build-env/
ls -altr

total 8
drwxr-xr-x 3 root root  29 Jan 31 21:04 ..
-rw-r--r-- 1 root root  79 Jan 31 21:04 README.md
drwxr-xr-x 8 root root 152 Jan 31 21:04 .git
-rw-r--r-- 1 root root 296 Jan 31 21:04 Dockerfile
drwxr-xr-x 3 root root  50 Jan 31 21:04 .


In [64]:
docker build -t $USER/cowsay-build-env .
docker images

Cloning into 'cowsay-build-env'...
remote: Counting objects: 12, done.
remote: Total 12 (delta 0), reused 0 (delta 0), pack-reused 12
Unpacking objects: 100% (12/12), done.

Step 1 : FROM ubuntu:trusty
 ---> 3876b81b5a81
Step 2 : RUN apt-get update -q &&     apt-get install -y -q --no-install-recommends         ca-certificates         cowsay         git         make
 ---> Running in f2c9615d0c8b
Ign http://archive.ubuntu.com trusty InRelease
Get:1 http://archive.ubuntu.com trusty-updates InRelease [64.4 kB]
Get:2 http://archive.ubuntu.com trusty-security InRelease [64.4 kB]
Hit http://archive.ubuntu.com trusty Release.gpg
Hit http://archive.ubuntu.com trusty Release
Get:3 http://archive.ubuntu.com trusty-updates/main Sources [311 kB]
Get:4 http://archive.ubuntu.com trusty-updates/restricted Sources [5219 B]
Get:5 http://archive.ubuntu.com trusty-updates/universe Sources [185 kB]
Get:6 http://archive.ubuntu.com trusty-updates/main amd64 Packages [865 kB]
Get:7 http://archive.ubuntu.com 

In [65]:
docker run -t $USER/cowsay-build-env

In [1]:
#docker run -it $USER/cowsay-build-env /bin/bash

In [7]:
cat Dockerfile 

FROM ubuntu:trusty

RUN apt-get update -q && \
    apt-get install -y -q --no-install-recommends \
        ca-certificates \
        cowsay \
        git \
        make

RUN ln -s /usr/games/cowsay /usr/bin

CMD git clone https://github.com/matsen/p-neq-np.git && \
    cd p-neq-np && \
    make


In [8]:
cat > Dockerfile <<EOF
FROM ubuntu:trusty

RUN apt-get update -q && \
    apt-get install -y -q --no-install-recommends \
        ca-certificates \
        cowsay \
        git \
        make \
        nyancat

RUN ln -s /usr/games/cowsay /usr/bin

CMD git clone https://github.com/matsen/p-neq-np.git && \
    cd p-neq-np && \
    make
EOF

In [9]:
docker build -t $USER/cowsay-build-env .
docker images


Step 1 : FROM ubuntu:trusty
 ---> 3876b81b5a81
Step 2 : RUN apt-get update -q &&     apt-get install -y -q --no-install-recommends         ca-certificates         cowsay         git         make         nyancat
 ---> Running in 71b23b53b79d
Ign http://archive.ubuntu.com trusty InRelease
Get:1 http://archive.ubuntu.com trusty-updates InRelease [64.4 kB]
Get:2 http://archive.ubuntu.com trusty-security InRelease [64.4 kB]
Hit http://archive.ubuntu.com trusty Release.gpg
Hit http://archive.ubuntu.com trusty Release
Get:3 http://archive.ubuntu.com trusty-updates/main Sources [311 kB]
Get:4 http://archive.ubuntu.com trusty-updates/restricted Sources [5219 B]
Get:5 http://archive.ubuntu.com trusty-updates/universe Sources [185 kB]
Get:6 http://archive.ubuntu.com trusty-updates/main amd64 Packages [865 kB]
Get:7 http://archive.ubuntu.com trusty-updates/restricted amd64 Packages [23.4 kB]
Get:8 http://archive.ubuntu.com trusty-updates/universe amd64 Packages [434 kB]
Get:9 http://archive.ubunt

In [10]:
docker run -t $USER/cowsay-build-env

#### How to push to Docker Hub from Jupyter??


# Hypriot example

Extend it with different backends
- Node
- Python server
- ??

# Example
<a href="#TOP">TOP</a>

This example heavily inspired from this article [microservice with rails and Docker](http://blog.giantswarm.io/getting-started-with-microservices-using-ruby-on-rails-and-docker/) ... but not complete, too cloudy ...

<font size=+1 color="#77f">
<b>The goal of this step is ...</b>   
</font>

In [ ]:
mkdir -p /root/src/nginx

cd /root/src/nginx

cat > Dockerfile <<EOF
FROM nginx

RUN rm -rf /usr/share/nginx/html
COPY public /usr/share/nginx/html
COPY config/deploy/nginx.conf /etc/nginx/conf.d/default.conf
EOF

A Dockerfile always starts with the FROM instruction. It tells Docker, which image is the base to be used. Additionally, we will just COPY the public folder and the configuration into the image. As I said before containers should be treated as immutable and due to this, we need to create a new image whenever we change the assets. The nginx config itself is pretty straight forward:

In [58]:
cat > 03_nginx_config.conf <<EOF
server {
    listen       80;
    server_name  localhost;

    location / {
        root      /usr/share/nginx/html;
        index     index.html index.htm;
        try_files $uri/index.html $uri.html $uri @upstream;
    }

    location @upstream {
        proxy_pass http://rails-app:8080;
    }
}
EOF

In [60]:
cat > Dockerfile.rails <<EOF
FROM ruby:2.1.5

# throw errors if Gemfile has been modified since Gemfile.lock
RUN bundle config --global frozen 1

RUN mkdir -p /usr/src/app
WORKDIR /usr/src/app

COPY Gemfile /usr/src/app/
COPY Gemfile.lock /usr/src/app/

RUN bundle install --deployment

COPY . /usr/src/app/

ENV RAILS_ENV production

EXPOSE 8080

CMD ["/usr/src/app/bin/rails", "server", "-p", "8080"]
EOF

In [62]:
cat > rake_tasks.rb <<EOF
namespace :docker do
  task :build => ['docker:build:web', 'docker:build:app', 'docker:build:worker', 'assets:clobber']

  namespace :build do
    task :web => ['assets:precompile', 'assets:clean'] do
      sh 'ln -snf Dockerfile.web Dockerfile'
      sh 'sudo docker build -t "registry.giantswarm.io/yoshida/gh-recommender-web" .'
      sh 'rm -f Dockerfile'
    end
    
    task :app => ['assets:precompile', 'assets:clean'] do
      sh 'ln -snf Dockerfile.app Dockerfile'
      sh 'sudo docker build -t "registry.giantswarm.io/yoshida/gh-recommender-app" .'
      sh 'rm -f Dockerfile'
    end

    task :worker do
      sh 'ln -snf Dockerfile.worker Dockerfile'
      sh 'sudo docker build -t "registry.giantswarm.io/yoshida/gh-recommender-worker" .'
      sh 'rm -f Dockerfile'
    end
  end
end
EOF

In [63]:
ls -altr /root/src/nginx

total 20
drwxr-xr-x 5 root root  46 Jan 31 20:54 ..
-rw-r--r-- 1 root root 140 Jan 31 20:55 Dockerfile
-rw-r--r-- 1 root root 287 Jan 31 20:57 01_nginx_config.conf
-rw-r--r-- 1 root root 287 Jan 31 20:57 03_nginx_config.conf
-rw-r--r-- 1 root root 374 Jan 31 20:59 Dockerfile.rails
-rw-r--r-- 1 root root 802 Jan 31 21:00 rake_tasks.rb
drwxr-xr-x 2 root root 120 Jan 31 21:00 .


One thing needs to be mentioned though: Where does this rails-app hostname come from? Docker will provide information about linked (we will explain later what that is) containers in two ways: A bunch of environment variables and entries in the /etc/hosts file. In this case we’re gonna use the entry from the /etc/hosts.

total 4
drwxr-xr-x 5 root root  46 Jan 31 20:54 ..
drwxr-xr-x 2 root root  23 Jan 31 20:55 .
-rw-r--r-- 1 root root 140 Jan 31 20:55 Dockerfile


# Rails Example with Compose
<a href="#TOP">TOP</a>

This example heavily inspired from this article [Building Microservices with Docker and the Rails API gem](https://medium.com/connect-the-dots/building-microservices-with-docker-and-the-rails-api-gem-2a463862f5d)

<font size=+1 color="#77f">
<b>The goal of this step is to have hands-on experience with Compose ...</b> 
<br/>
It is recommended to use [yamllint](http://www.yamllint.com/) to validate your YAML file - because it's easy to make mistakes in YAML, and Compose is picky.
</font>

In [36]:
cd /root
mkdir -p src/railsapi
cd src/railsapi
pwd
touch Dockerfile docker-compose.yml Gemfile Gemfile.lock

/root/src/railsapi


In [37]:
cat > Dockerfile <<EOF

FROM ruby:2.3.0

RUN apt-get update -qq && apt-get install -y build-essential libmysqlclient-dev

RUN mkdir /railsapi

WORKDIR /railsapi

ADD Gemfile /railsapi/Gemfile

ADD Gemfile.lock /railsapi/Gemfile.lock

RUN bundle install

ADD . /railsapi

EOF


See [References](#References) section below for information on *Compose*



In [38]:
cat > docker-compose.yml <<EOF

version: 2
services:
  db:
   image: mysql:latest
   ports:
     - 3306:3306
   environment:
     MYSQL_ROOT_PASSWORD: mypassword
   
  web:
    build: .
    command: puma
    ports:
      - 9292:9292
    links:
      - db
    volumes:
      - .:/railsapi

EOF

In [39]:
docker-compose build

db uses an image, skipping
Building web
Step 1 : FROM ruby:2.3.0
 ---> 70578fbdd1a4
Step 2 : RUN apt-get update -qq && apt-get install -y build-essential libmysqlclient-dev
 ---> Using cache
 ---> 542437588210
Step 3 : RUN mkdir /railsapi
 ---> Using cache
 ---> 458b06826730
Step 4 : WORKDIR /railsapi
 ---> Using cache
 ---> 972ec34e499e
Step 5 : ADD Gemfile /railsapi/Gemfile
 ---> Using cache
 ---> 60947231435f
Step 6 : ADD Gemfile.lock /railsapi/Gemfile.lock
 ---> 53c629b24a97
Removing intermediate container 1aac5f737465
Step 7 : RUN bundle install
 ---> Running in c7e1111ea35d
Fetching gem metadata from https://rubygems.org/...........
Fetching version metadata from https://rubygems.org/...
Fetching dependency metadata from https://rubygems.org/..
Installing rake 10.5.0
Installing i18n 0.7.0
Using json 1.8.3
Installing minitest 5.8.4
Installing thread_safe 0.3.5
Installing builder 3.2.2
Installing erubis 2.7.0
Installing mini_portile2 2.0.0
Installing rack 1.6.4
Installing mime-type

In [40]:
cat > Gemfile <<EOF

source 'https://rubygems.org'

gem 'rails', '4.2.5'
gem 'rails-api', '0.4.0'
gem 'mysql2'
gem 'puma'

# Use ActiveModel has_secure_password
# gem 'bcrypt', '~> 3.1.7'

# Use Capistrano for deployment
# gem 'capistrano-rails', group: :development

# Use Rack CORS for handling Cross-Origin Resource Sharing (CORS), making cross-origin AJAX possible
# gem 'rack-cors'

# Use ActiveModelSerializers to serialize JSON responses
gem 'active_model_serializers', '~> 0.10.0.rc3'

group :development, :test do
  # Call 'byebug' anywhere in the code to stop execution and get a debugger console
  gem 'byebug'
end
 
group :development do # Spring speeds up development by keeping your application running in the background. Read more: https://github.com/rails/spring
  gem 'spring'
end

EOF

In [41]:
ls -altr

total 32
-rw-r--r-- 1 root root  478 Jan 31 19:34 README.rdoc
-rw-r--r-- 1 root root  249 Jan 31 19:34 Rakefile
-rw-r--r-- 1 root root  474 Jan 31 19:34 .gitignore
-rw-r--r-- 1 root root  153 Jan 31 19:34 config.ru
drwxr-xr-x 2 root root  147 Jan 31 19:34 .
drwxr-xr-x 4 root root   34 Jan 31 19:45 ..
-rw-r--r-- 1 root root 2928 Jan 31 20:01 Gemfile.lock
-rw-r--r-- 1 root root  248 Jan 31 20:01 Dockerfile
-rw-r--r-- 1 root root  258 Jan 31 20:02 docker-compose.yml
-rw-r--r-- 1 root root  779 Jan 31 20:10 Gemfile


Now let's build our image

In [42]:
docker-compose build

db uses an image, skipping
Building web
Step 1 : FROM ruby:2.3.0
 ---> 70578fbdd1a4
Step 2 : RUN apt-get update -qq && apt-get install -y build-essential libmysqlclient-dev
 ---> Using cache
 ---> 542437588210
Step 3 : RUN mkdir /railsapi
 ---> Using cache
 ---> 458b06826730
Step 4 : WORKDIR /railsapi
 ---> Using cache
 ---> 972ec34e499e
Step 5 : ADD Gemfile /railsapi/Gemfile
 ---> Using cache
 ---> 60947231435f
Step 6 : ADD Gemfile.lock /railsapi/Gemfile.lock
 ---> Using cache
 ---> 53c629b24a97
Step 7 : RUN bundle install
 ---> Using cache
 ---> 2611f8731623
Step 8 : ADD . /railsapi
 ---> Using cache
 ---> 7159710bbffe
Successfully built 7159710bbffe


In [43]:
docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
railsapi_web        latest              7159710bbffe        6 minutes ago       813.8 MB
lab/basic           latest              5477a1cdda04        11 minutes ago      689.1 MB
<none>              <none>              a192650246e9        11 minutes ago      689.1 MB
<none>              <none>              a7d63cd9ba3c        12 minutes ago      689.1 MB
<none>              <none>              8855922a993d        46 minutes ago      813.8 MB
<none>              <none>              bef232158200        48 minutes ago      746.3 MB
rails_web           latest              5152cec9abc7        53 minutes ago      813.8 MB
<none>              <none>              349e9cce0641        About an hour ago   813.8 MB
<none>              <none>              96b443625f76        About an hour ago   813.8 MB
<none>              <none>              7d9bce362118        About an hour ago   813.8 MB
<none>              <none

In [44]:
docker-compose run web rails-api new .

       exist  
   identical  README.rdoc
   identical  Rakefile
   identical  config.ru
   identical  .gitignore
    conflict  Gemfile
Overwrite /railsapi/Gemfile? (enter "h" for help) [Ynaqdh] 


In [46]:
cat > database.yml <<EOF
development:
 adapter: mysql2
 encoding: utf8
 reconnect: false
 database: inventory_manager_dev
 pool: 5
 username: root
 password: mypassword
 host: db
test:
 adapter: mysql2
 encoding: utf8
 reconnect: false
 database: inventory_manager_test
 pool: 5
 username: root
 password: mypassword
 host: db
EOF

In [48]:
docker-compose up web

railsapi_db_1 is up-to-date
Creating railsapi_web_1
Attaching to railsapi_web_1
web_1 | Puma starting in single mode...
web_1 | * Version 2.16.0 (ruby 2.3.0-p0), codename: Midwinter Nights Trance
web_1 | * Min threads: 0, max threads: 16
web_1 | * Environment: development
web_1 | config.ru:3:in `require': cannot load such file -- /railsapi/config/environment (LoadError)
web_1 | 	from config.ru:3:in `block in <main>'
web_1 | 	from /usr/local/bundle/gems/rack-1.6.4/lib/rack/builder.rb:55:in `instance_eval'
web_1 | 	from /usr/local/bundle/gems/rack-1.6.4/lib/rack/builder.rb:55:in `initialize'
web_1 | 	from config.ru:in `new'
web_1 | 	from config.ru:in `<main>'
web_1 | 	from /usr/local/bundle/gems/rack-1.6.4/lib/rack/builder.rb:49:in `eval'
web_1 | 	from /usr/local/bundle/gems/rack-1.6.4/lib/rack/builder.rb:49:in `new_from_string'
web_1 | 	from /usr/local/bundle/gems/rack-1.6.4/lib/rack/builder.rb:40:in `parse_file'
web_1 | 	from /usr/local/bundle/gems/puma-2.16.0/lib/puma/configuration.rb

In [49]:
curl http://localhost:80

curl: (7) Failed to connect to localhost port 80: Connection refused


# Building Docker with Docker
<a href="#TOP">TOP</a>

A major advantage of Docker is to simplify build environments.

Let's look at how we can build the Docker engine client/daemon binary without having to explicitly install a development environment.

<font size=+1 color="#77f">
<b>The goal of this step is simply to show the ease with which we can build Docker, thanks to Docker itself.</b>   
</font>

We do not make particular use of the built image.

The process involves the following steps, several of which have already been performed so as to prevent excessive network utilisation during the lab.  Nevertheless all steps are  described here so that you can see just how easy it is to build Docker from scratch:
- Install make
- Clone the Docker source code
- Checkout the same code revision as our current Docker binary (client and daemon)
- Build the code - which pulls the docker-dev image containing the required version of the Go compiler
- Run the executeable to demonstrate it is correct

#### Installing make

In your environment we have already installed the make package, but no compiler using yum:

    yum install make

#### Cloning the Docker source code

We have already downloaded the Docker source code from github as follows:

    mkdir -p /root/src/docker
    cd /root/src/docker
    git clone https://github.com/docker/docker .

To build Docker we simply have to build using the

    make build
    
command.

#### Checkout the source code revision corresponding to our installed Docker Engine

If we build the latest sources this may not be compatible with our installed Docker version.

This is the case.  We have 1.10.0-rc2 installed, which has API version 22, but the current github source is 1.10.0-dev which has changed to API version 23.  So if we build this we find that we cannot use this client to communicate with the installed daemon.

So let's checkout the code for 1.10.0-rc2.

At the time of writing this is the latest release(candidate) of the Docker engine.
We can obtain that version of the source code by referring to the releases page https://github.com/docker/docker/releases
and selecting the SHA1 hash of build 1.10.0-rc2 

    git checkout c1cdc6e



#### Build the code - which pulls the docker-dev image containing the required version of the Go compiler

We can build the code as follows:

    make build
    
We have run 'make build' already, so the docker-dev image has already been downloaded (again to prevent excessive network traffic).  The docker-dev image includes the required go compiler and other build tools.

Run 'make build' again and you will see a standard build process and finally where it places the compiled binary

#### Run the executeable to demonstrate it is correct

In preparation for the lab we built from the latest source (not the c1cdc6e version we checked out).

Run this build as follows to see that it is not compatible with the installed binary (/usr/bin/docker).
We see that this binary has version 1.10.0-dev and API version 1.23 but that this cannot communicate with our installed binary which has API version 1.22.

In [ ]:
cd /root/src/docker; ls -altr bundles/1.10.0-dev/binary/docker-1.10.0-dev; ./bundles/1.10.0-dev/binary/docker version

But if we run our new build - as follows - created from revision c1cdc6e of the source code (corresponding to Docker version 1.10.0-rc2) we see that it has the correct version, with the same API version and can interrogate the server.


In [ ]:
cd /root/src/docker; ls -altr bundles/1.10.0-rc2/binary/docker-1.10.0-rc2; ./bundles/1.10.0-rc2/binary/docker version

# Docker Builds Composition and Orchestration
  - Compose
    - First guide through an n-Tier linked container example
      - build
      - run with linking
    - Now just compose

  - Swarm? / Mesos? / Kubernetes?
  - Weave?

# Future Lab Ideas
- #### Docker Container Services (**FUTURE e.g. for TES?**)
  - Amazon ECS
  - Digital Ocean
  - MS Azure (**best**)
  - OpenStack Magnum
    - Helion OpenStack (**best**)

- #### Docker Continuous Integration (**FUTURE e.g. for TES?**)
  - Continuous Integration

## Demo
[TOP](#TOP)

Demo duration (mins):




# References
[TOP](#TOP)

- [Dockerfile Reference](https://docs.docker.com/engine/reference/builder/)

- [Compose file documentation](https://docs.docker.com/compose/compose-file/)

- [Compose file reference](https://github.com/docker/compose/blob/1.6.0-rc1/docs/compose-file.md)
